# Introduction
Ce rapport présente une analyse des données nutritionnelles à l'aide de PySpark, un framework de traitement des données distribuées. L'objectif de cette analyse est de nettoyer et de traiter les données sur les produits alimentaires, les régimes alimentaires et les utilisateurs pour générer un menu hebdomadaire équilibré.

# 1) Récupération des données
Pour récupérer les données OpenFoodFacts, nous avons exploité la source de données accessible à l'adresse https://fr.openfoodfacts.org/data. À partir de cette source, nous avons téléchargé le fichier CSV contenant les informations sur les produits alimentaires. Ensuite, en utilisant PySpark, nous avons chargé et traité ces données dans notre environnement de développement Eclipse. De plus, nous avons affiché la structure racine du fichier ainsi que le nombre de lignes et de colonnes pour évaluer le volume de donnée

In [2]:
# Importation des bibliothéques
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, regexp_replace
from pyspark.sql.types import IntegerType

# 1.1) Chargement des données
Les données sont chargées à partir de fichiers CSV locaux. Trois ensembles de données sont utilisés:

Données des produits alimentaires provenant d'Openfoodfacts
Données sur les régimes nutritionnels
Données des utilisateurs et de leurs régimes alimentaires

In [4]:
from pyspark.sql import SparkSession
# Création de la session Spark
spark = SparkSession.builder.appName("ERP-OpenProductFact").master("local[1]").getOrCreate()

# Configuration pour afficher toutes les colonnes
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
filePath="/home/sokhna/Téléchargements/en.openfoodfacts.org.products.csv"
# Charger les données CSV dans un DataFrame Spark
df_openfoodfacts = spark.read.options(header=True, inferSchema=True, delimiter='\t').csv(filePath)

df_openfoodfacts

24/03/01 14:27:35 WARN Utils: Your hostname, sokhna-GF75-Thin-10SCSR resolves to a loopback address: 127.0.1.1; using 172.20.10.4 instead (on interface enx3ecd362a2b81)
24/03/01 14:27:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/01 14:27:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/01 14:28:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_en,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutrition_data,additives_n,additives,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,food_groups,food_groups_tags,food_groups_en,states,states_tags,states_en,brand_owner,ecoscore_score,ecoscore_grade,nutrient_levels_tags,product_quantity,owner,data_quality_errors_tags,unique_scans_n,popularity_tags,completeness,last_image_t,last_image_datetime,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,unsaturated-fat_100g,monounsaturated-fat_100g,omega-9-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,omega-6-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,added-sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,erythritol_100g,fiber_100g,soluble-fiber_100g,insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,added-salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
225.0,http://world-en.o...,nutrinet-sante,1623855208,2021-06-16 16:53:28,1692101569,2023-08-15 14:12:49,digg,1707747939,2024-02-12 15:25:39,jeunes pousses,NULL,NULL,NULL,NULL,NULL,NULL,NULL,endives,endives,Aliments et boiss...,en:plant-based-fo...,Plant-based foods...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,en:france,en:france,France,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,unknown,NULL,Fruits and vegeta...,Vegetables,en:vegetables,en:fruits-and-veg...,Fruits and vegeta...,en:to-be-complete...,en:to-be-complete...,"To be completed,N...",NULL,79.0,b,NULL,NULL,NULL,NULL,NULL

# 1.2) Informations sur les données
Le fichier comporte 3085838 lignes et 206 colonnes. Le type de données varie entre type textuelle ou numérique.
On a défini aussi une fonction **column_info** qui permet de donner les caractéristique d'une colonne spécifique.On a pris la colonne **code** comme exemple pour la tester.

In [5]:
# Affichage du schéma des données
df_openfoodfacts.printSchema()

root
 |-- code: double (nullable = true)
 |-- url: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- created_t: integer (nullable = true)
 |-- created_datetime: timestamp (nullable = true)
 |-- last_modified_t: integer (nullable = true)
 |-- last_modified_datetime: timestamp (nullable = true)
 |-- last_modified_by: string (nullable = true)
 |-- last_updated_t: integer (nullable = true)
 |-- last_updated_datetime: timestamp (nullable = true)
 |-- product_name: string (nullable = true)
 |-- abbreviated_product_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- packaging: string (nullable = true)
 |-- packaging_tags: string (nullable = true)
 |-- packaging_en: string (nullable = true)
 |-- packaging_text: string (nullable = true)
 |-- brands: string (nullable = true)
 |-- brands_tags: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- categories_tags: string (nullable = true)
 |-- 

In [6]:
# Nombre total de lignes
total_rows = df_openfoodfacts.count()
print("Nombre total de lignes :", total_rows)
# Nombre total de colonnes
total_columns = len(df_openfoodfacts.columns)
print("Nombre total de colonnes :", total_columns)

Nombre total de lignes : 3085838
Nombre total de colonnes : 206


In [7]:
# Demande information pour une colonne spécifique
def column_info(column_name):
  
    # Vérification si la colonne existe
    if column_name in df_openfoodfacts.columns:
        # Affichage des informations de la colonne
        print("Informations pour la colonne :", column_name)
        df_openfoodfacts.select(column_name).describe().show()
    else:
        print("La colonne", column_name, "n'existe pas dans le DataFrame.")

# Exemple la colonne code
column_info("code")

Informations pour la colonne : code


+-------+--------------------+
|summary|                code|
+-------+--------------------+
|  count|             3085838|
|   mean|4.848224249563991E49|
| stddev|6.072651250257753E52|
|    min|                 1.0|
|    max|8.450851484235123E55|
+-------+--------------------+



# 2) Nettoyage des données

# 2.1) Sélection des Colonnes Utiles :

Initialement, nous avons entrepris de nettoyer les données en éliminant toutes les lignes comportant des valeurs NULL. Cependant, nous avons rapidement constaté que l'intégralité du fichier contenait des valeurs NULL, aboutissant ainsi à un tableau vide. Nous avons donc dû explorer une autre approche. Par conséquent, nous avons opté pour la sélection des colonnes pertinentes. Nous avons identifié et conservé uniquement les colonnes jugées essentielles pour répondre à nos besoins. Celles-ci incluent le code produit, le nom du produit, le pays d'origine, le grade nutriscore et les informations nutritionnelles telles que l'énergie, les lipides, les glucides, les protéines et le sel.

# 2.2) Suppression des Données Manquantes ou Incomplètes :

Nous avons employé des transformations Spark pour filtrer les lignes du DataFrame relatives aux produits alimentaires pour lesquels des informations essentielles étaient absentes ou incomplètes. Cela inclut les produits sans nom ou sans données nutritionnelles significatives. Nous avons choisi de conserver la valeur "unknown" pour le nutriscore, car nous avons jugé que cette information n'était pas indispensable mais pouvait être bénéfique si elle était disponible.

# 2.3) Filtrage des Valeurs Aberrantes :

En exploitant les capacités de traitement distribué d'Apache Spark, nous avons identifié et éliminé les valeurs aberrantes dans les données, en particulier pour les composantes nutritionnelles telles que les lipides, les glucides, les protéines, etc. Les valeurs extrêmes ou peu plausibles ont été filtrées pour garantir la qualité des données. Apres filtrations on retrouve avec 802700 lignes et 9 colonnes.


In [20]:
# Sélection des colonnes à conserver
columns_to_keep = ["code", "product_name", "countries_en", "nutriscore_grade", "energy_100g", "fat_100g",
                     "carbohydrates_100g", "proteins_100g", "salt_100g"]

# Sélection des colonnes dans le DataFrame
df_openfoodfacts = df_openfoodfacts.select(*colonnes_a_garder)


In [21]:
    # Suppression les lignes avec des valeurs manquantes ou mal formatées
    df_openfoodfacts = open_food_facts_data.filter(
        col("product_name").isNotNull() &
        col("countries_en").isNotNull() &
        col("nutriscore_grade").isNotNull() &
        col("energy_100g").isNotNull() &
        col("fat_100g").isNotNull() &
        col("carbohydrates_100g").isNotNull() &
        col("proteins_100g").isNotNull() &
        col("salt_100g").isNotNull()
    )

In [42]:
# Convertir les colonnes pertinentes en type double pour effectuer des calculs
for column in columns_to_keep:
    if df_openfoodfacts.schema[column].dataType.simpleString() == "string":
        continue
    df_openfoodfacts = df_openfoodfacts.withColumn(column, col(column).cast("double"))


In [43]:
# Définir les bornes acceptables pour chaque colonne
min_values = [1, 0, 0, 0, 0, 0]  # bornes minimales
max_values = [900, 100, 100, 100, 100, 100]  # bornes maximales


In [44]:
# Filtrage des lignes contenant des valeurs aberrantes
for i, column in enumerate(columns_to_keep[4:], start=4):
    min_value = min_values[i - 4]
    max_value = max_values[i - 4]
    df_openfoodfacts = df_openfoodfacts.filter((col(column) >= min_value) & (col(column) <= max_value))


In [46]:
 # Nombre de lignes après filtrage
row_count_after = df_openfoodfacts.count()
# Nombre de colonnes après filtrage
column_count_after = len(df_openfoodfacts.columns)
print(f"Après le nettoyage, le dataset compte {row_count_after} lignes et {column_count_after} colonnes.")


Après le nettoyage, le dataset compte 802700 lignes et 9 colonnes.


In [47]:
# Affichage des 20 premières lignes du tableau
df_openfoodfacts.show()

+-----------+--------------------+------------+----------------+-----------+--------+------------------+-------------+---------+
|       code|        product_name|countries_en|nutriscore_grade|energy_100g|fat_100g|carbohydrates_100g|proteins_100g|salt_100g|
+-----------+--------------------+------------+----------------+-----------+--------+------------------+-------------+---------+
|2.6772226E7|                Skyr|      France|               a|      238.0|     0.2|               3.9|         10.0|     0.09|
|      123.0|Sauce Sweety chil...|      France|         unknown|       88.0|     0.0|               4.8|          0.2|     2.04|
|      178.0|           Mini coco|      France|         unknown|      251.0|     3.0|              10.0|          2.0|     1.15|
|       80.0|    Pur jus de pomme|      France|               d|      264.0|     0.0|              15.5|          0.2|      0.0|
|      949.0|Salade de carotte...|      France|               a|      134.0|     0.3|            

# 3) Création des fichiers CSV utilisateurs et régimes
Nous avons créé deux fichiers :

# a. regimes_nutritionnels: 
ce fichier représente une table de référence pour différents régimes alimentaires avec des seuils recommandés pour certaines valeurs nutritionnelles. Chaque ligne correspond à un régime spécifique avec des valeurs maximales recommandées pour les nutriments tels que les glucides, les protéines, les lipides et les calories.
  Ce fichier compose les colonnes suivantes:
  
  - **régime**: Cette colonne représente le nom du régime alimentaire. Chaque ligne correspond à un régime spécifique, par exemple, "FODMAP", "Mediterraneen", "Paleo", etc.
  - **max_glucides_g**: Cette colonne indique la quantité maximale recommandée de glucides (en grammes) par jour pour une personne suivant ce régime alimentaire.
  - **max_proteines_g**: Cette colonne indique la quantité maximale recommandée de protéines (en grammes) par jour pour une personne suivant ce régime alimentaire.
  - **max_lipides_g**: Cette colonne indique la quantité maximale recommandée de lipides (en grammes) par jour pour une personne suivant ce régime alimentaire.
  - **max_calories**: Cette colonne indique la quantité maximale recommandée de calories par jour pour une personne suivant ce régime alimentaire.
  
  Ce fichier est utilisé dans notre application Spark pour définir les critères de seuils lors de la génération d'un menu alimentaire personnalisé en fonction du régime alimentaire d'un utilisateur. Les colonnes de ce fichier sont sélectionnées et utilisées lors de la jointure avec les données des utilisateurs pour créer un menu personnalisé respectant les recommandations nutritionnelles spécifiques à chaque régime.
# b. utilisateurs_regimes: 
Le fichier représente une table d'utilisateurs avec des informations telles que l'identifiant utilisateur, l'âge, le sexe, le poids et le régime alimentaire suivi par chaque utilisateur.
  Les colonnes sont les suivantes :
  
  - **utilisateur_id**: Cette colonne représente un identifiant unique pour chaque utilisateur. Chaque ligne a un identifiant utilisateur différent.
  - **age**: Cette colonne indique l'âge de l'utilisateur.
  - **sexe**: Cette colonne représente le sexe de l'utilisateur, généralement "M" pour masculin et "F" pour féminin.
  - **poids**: Cette colonne indique le poids de l'utilisateur.
  - **regime_alimentaire**: Cette colonne indique le régime alimentaire suivi par l'utilisateur. Les valeurs peuvent être, par exemple, "Vegetarien", "Cetogene", "Vegan", etc.
  
  Ces informations sur les utilisateurs sont utilisées dans notre application Spark pour personnaliser les menus alimentaires en fonction des préférences et des besoins nutritionnels de chaque utilisateur. Lorsque nous générons le menu hebdomadaire, nous pouvons utiliser ces informations pour appliquer des filtres spécifiques basés sur le régime alimentaire de chaque utilisateur, en veillant à respecter les seuils recommandés pour les nutriments spécifiés dans le fichier de régimes alimentaires.


In [48]:
    # Chargement des données des utilisateurs depuis le lien CSV
    utilisateurs_data = spark.read.format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .option("encoding", "UTF-8") \
        .load("/home/sokhna/Téléchargements/utilisateurs_regimes.csv")
utilisateurs_data

utilisateur_id,age,sexe,poids,regime_alimentaire
1,30,M,70,Vegetarien
2,25,F,65,Cetogene
3,40,M,80,Vegan
4,35,F,60,Mediterraneen
5,28,M,75,DASH
6,45,F,68,FODMAP
7,32,M,85,Low_Carb
8,27,F,62,Paleo
9,38,M,78,Weight_Watchers
10,29,F,70,Zone


In [49]:
# Chargement des données régimes depuis le lien CSV
regimes_data = spark.read.format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .option("encoding", "UTF-8") \
        .load("/home/sokhna/Téléchargements/regimes_data.csv")
regimes_data

regime,max_glucides_g,max_proteines_g,max_lipides_g,max_calories
FODMAP,50,30,20,200
Mediterraneen,50,30,20,201
Paleo,40,25,35,1800
Vegetarien,55,25,25,2000
Cetogene,10,70,80,1500
Vegan,50,30,20,2000
DASH,60,20,25,2200
Weight_Watchers,40,25,25,1800
Sans_gluten,50,30,20,208
Alcalin,50,20,30,2000


# 4) Intégration des Informations Utilisateurs avec les Seuils des Régimes Alimentaires

Dans cette section, nous intégrons les informations des utilisateurs avec les seuils des régimes alimentaires. Nous effectuons une jointure entre les données des utilisateurs et celles des régimes alimentaires en utilisant la colonne "regime_alimentaire" comme clé de jointure. Les colonnes sélectionnées incluent l'identifiant de l'utilisateur, le régime alimentaire, ainsi que les seuils maximums de glucides, de protéines, de lipides et de calories.


# Génération Aléatoire du Menu Hebdomadaire

Dans cette étape, nous générons aléatoirement un menu équilibré pour chaque jour de la semaine en fonction des produits alimentaires disponibles dans notre ensemble de données OpenFoodFacts. Nous filtrons les données pour nous assurer que les valeurs nutritionnelles essentielles telles que l'énergie, les lipides, les glucides, les protéines et le sel ne sont pas nulles.

Pour chaque jour de la semaine, nous sélectionnons aléatoirement 7 produits alimentaires à partir de notre ensemble de données OpenFoodFacts, en garantissant qu'ils sont représentatifs des besoins nutritionnels des utilisateurs. Nous ajoutons ensuite ces sélections à notre menu hebdomadaire.

# Stockage du Menu Hebdomadaire

Une fois le menu hebdomadaire généré, nous le stockons dans un fichier CSV à l'emplacement spécifié sur le système de fichiers local. De plus, nous créons une table dans le Data Warehouse (DWH) pour stocker ce menu. Une autre table est créée pour établir un lien entre le menu hebdomadaire et l'utilisateur final.


Ce rapport résume le processus de génération du menu hebdomadaire à partir des données OpenFoodFacts, des informations sur les utilisateurs et des régimes alimentaires, ainsi que les étapes suivies pour stocker les résultats dans un entrepôt de données.

In [62]:
  # Intégration des informations des utilisateurs avec les seuils des régimes alimentaires
menu_personnalise = utilisateurs_data.join(regimes_data, utilisateurs_data["regime_alimentaire"] == regimes_data["regime"]) \
        .select("utilisateur_id", "regime_alimentaire", "max_glucides_g", "max_proteines_g", "max_lipides_g",
                "max_calories")

menu_personnalise.show()



+--------------+--------------------+--------------+---------------+-------------+------------+
|utilisateur_id|  regime_alimentaire|max_glucides_g|max_proteines_g|max_lipides_g|max_calories|
+--------------+--------------------+--------------+---------------+-------------+------------+
|             6|              FODMAP|            50|             30|           20|         200|
|             4|       Mediterraneen|            50|             30|           20|         201|
|             8|               Paleo|            40|             25|           35|        1800|
|             1|          Vegetarien|            55|             25|           25|        2000|
|             2|            Cetogene|            10|             70|           80|        1500|
|             3|               Vegan|            50|             30|           20|        2000|
|             5|                DASH|            60|             20|           25|        2200|
|             9|     Weight_Watchers|   

In [63]:
# Générer aléatoirement un menu sur une semaine
menu_hebdomadaire = df_openfoodfacts.filter(col("energy_100g").isNotNull()) \
                                             .filter(col("fat_100g").isNotNull()) \
                                             .filter(col("carbohydrates_100g").isNotNull()) \
                                             .filter(col("proteins_100g").isNotNull()) \
                                             .filter(col("salt_100g").isNotNull())
menu_hebdomadaire

code,product_name,countries_en,nutriscore_grade,energy_100g,fat_100g,carbohydrates_100g,proteins_100g,salt_100g
2.6772226E7,Skyr,France,a,238.0,0.2,3.9,10.0,0.09
123.0,Sauce Sweety chil...,France,unknown,88.0,0.0,4.8,0.2,2.04
178.0,Mini coco,France,unknown,251.0,3.0,10.0,2.0,1.15
80.0,Pur jus de pomme,France,d,264.0,0.0,15.5,0.2,0.0
949.0,Salade de carotte...,France,a,134.0,0.3,5.3,0.9,0.42
970.0,Fromage blanc aux...,France,unknown,540.0,4.9,16.3,4.4,0.25
112.0,Iogurt de cabra,Spain,unknown,305.0,4.3,4.3,3.3,0.1
1311.0,Salade de fruits ...,France,unknown,418.0,0.0,24.0,0.0,3.2
1328.0,Chouquettes x 30,France,unknown,657.0,21.0,39.0,7.8,1.5
1564.0,Fromage blanc pêche,France,unknown,410.0,3.3,7.6,4.6,0.027


In [64]:
# Répéter pour chaque jour de la semaine
for jour in range(1, 8):
        # Sélectionner aléatoirement des produits pour chaque jour
        menu_jour = menu_hebdomadaire.sample(False, 0.1).limit(7).withColumn("jour", lit(jour))
menu_jour

code,product_name,countries_en,nutriscore_grade,energy_100g,fat_100g,carbohydrates_100g,proteins_100g,salt_100g,jour
178.0,Mini coco,France,unknown,251.0,3.0,10.0,2.0,1.15,7
1311.0,Salade de fruits ...,France,unknown,418.0,0.0,24.0,0.0,3.2,7
2363.0,Suedois thon,France,unknown,678.0,3.7,20.7,9.1,0.194,7
5166.0,cuisse de poulet ...,France,c,824.0,13.0,3.0,17.0,1.1,7
15509.0,Baguette Niçois,France,unknown,812.0,5.6,27.7,9.6,1.1,7
1576.0,Pur jus de pomme,France,d,221.0,0.5,13.0,0.5,0.01,7
6.4079933414E10,Pechuga de pavo b...,Spain,unknown,322.0,1.0,2.0,15.0,1.8,7


In [65]:
# Affichage du menu de chaque jour
print("Voici le menu du jour :")
menu_jour.show()

Voici le menu du jour :
+---------------+--------------------+------------+----------------+-----------+--------+------------------+-------------+---------+----+
|           code|        product_name|countries_en|nutriscore_grade|energy_100g|fat_100g|carbohydrates_100g|proteins_100g|salt_100g|jour|
+---------------+--------------------+------------+----------------+-----------+--------+------------------+-------------+---------+----+
|          178.0|           Mini coco|      France|         unknown|      251.0|     3.0|              10.0|          2.0|     1.15|   7|
|         1311.0|Salade de fruits ...|      France|         unknown|      418.0|     0.0|              24.0|          0.0|      3.2|   7|
|         2363.0|        Suedois thon|      France|         unknown|      678.0|     3.7|              20.7|          9.1|    0.194|   7|
|         5166.0|cuisse de poulet ...|      France|               c|      824.0|    13.0|               3.0|         17.0|      1.1|   7|
|        1

In [ ]:
# Stockage du menu dans le DWH
menu_hebdomadaire_path = "/home/sokhna/Téléchargements/menu_hebdomadaire"
menu_hebdomadaire.write.format("csv") \
                     .option("header", "true") \
                     .option("delimiter", ",") \
                     .mode("overwrite") \
                     .save(menu_hebdomadaire_path)

In [77]:
# Création d'une table dans le DWH pour stocker le menu hebdomadaire
menu_hebdomadaire.createOrReplaceTempView("table_menu_hebdomadaire")

In [ ]:
spark.conf.set("spark.sql.catalogImplementation", "hive")

# Création d'une table pour lier le menu hebdomadaire à l'utilisateur final
sql_create_link_table = """
    CREATE TABLE IF NOT EXISTS table_lien_utilisateur_menu (
        utilisateur_id INT,
        menu_id INT,
        jour INT
    )
    USING PARQUET
    OPTIONS (provider "parquet")
"""
spark.sql(sql_create_link_table)


In [ ]:
# Ajout des informations de liaison dans la table du DWH
sql_insert_link_info = "INSERT INTO table_lien_utilisateur_menu VALUES (utilisateur_id, menu_id, jour)"
spark.sql(sql_insert_link_info)

In [ ]:
spark.stop()

# Conclusion
Ce rapport présente une analyse complète des données nutritionnelles à l'aide de PySpark. Le processus comprend le chargement, le nettoyage, l'intégration et la génération d'un menu hebdomadaire personnalisé pour chaque utilisateur. Les résultats obtenus peuvent être utilisés pour recommander des menus alimentaires équilibrés aux utilisateurs en fonction de leurs besoins nutritionnels.